# Tema 1 AM

##  Client Side

In [ ]:
# importare biblioteci
import sys
import os
import socket
from PySide6.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QMessageBox, QFileDialog
import pandas as pd
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from sklearn.metrics import confusion_matrix
import seaborn as sns
from docx import Document
from docx.shared import Inches

In [ ]:
# functiile programului
class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.first_button_clicked = False
        self.second_button_clicked = False
        self.initUI()

    def initUI(self):
        # Definim butoanele
        button1 = QPushButton('Citeste CSV', self)
        button2 = QPushButton('Afiseaza Graficul', self)
        button3 = QPushButton('Genereaza Raport Docx', self)

        # Setam dimensiunile butoanelor
        button1.setFixedSize(150, 50)
        button2.setFixedSize(150, 50)
        button3.setFixedSize(150, 50)

        # Definim layout-ul pentru butoane
        hbox_buttons = QHBoxLayout()
        hbox_buttons.addWidget(button1)
        hbox_buttons.addWidget(button2)
        hbox_buttons.addWidget(button3)

        vbox = QVBoxLayout()
        vbox.addLayout(hbox_buttons)

        # Canvas-ul pentru grafic
        self.canvas = FigureCanvas(Figure(figsize=(5, 3)))
        vbox.addWidget(self.canvas)

        # Conectam semnalul clicked al primului buton la slot-ul nostru
        button1.clicked.connect(self.readCSV)
        # Conectam semnalul clicked al celui de-al doilea buton la slot-ul nostru
        button2.clicked.connect(self.showGraph)
        # Conectam semnalul clicked al celui de-al treilea buton la slot-ul nostru
        button3.clicked.connect(self.generateDocx)

        # Setam layout-ul pentru fereastra principala
        self.setLayout(vbox)

        # Setam dimensiunile ferestrei (am ales o dimensiune mai mare ca sa se poata observa cat mai ok graficele)
        self.setGeometry(100, 100, 1500, 800)
        self.setWindowTitle('Interfata Tema 1 AM')

        # Afisam fereastra
        self.show()

    def readCSV(self):
        try:
            # Fereastra pentru alegerea fisierului CSV
            csv_file_path, _ = QFileDialog.getOpenFileName(self, "Alege fisierul CSV", "", "CSV Files (*.csv)")
            if not csv_file_path:  # daca nu se alege niciun fisier
                return

            # Citim fișierul CSV ales
            self.df = pd.read_csv(csv_file_path, delimiter=';', header=None,
                                  names=['Facial recognition results', 'True Label', 'Predicted Label', 'Matches'])
            # Afișăm un mesaj de succes (am comentat mesajele pentru butoane deoarece nu erau cerute in enunt - eu le-am folosit pt testare si am zis totusi sa le las)
            # QMessageBox.information(self, "Succes", "Fișierul CSV a fost citit cu succes!")
            self.first_button_clicked = True
        except Exception as e:
            # Dacă apare o excepție, afișăm un mesaj de eroare
            QMessageBox.critical(self, "Eroare", f"A apărut o eroare la citirea fișierului CSV: {str(e)}")

    def showGraph(self):
        if not self.first_button_clicked:
            QMessageBox.warning(self, "Atentie", "Te rog, apasa mai intai butonul pentru a citi fisierul CSV!")
            return

        # Filtram valorile NaN din Matches
        self.df['Matches'] = pd.to_numeric(self.df['Matches'], errors='coerce')

        # Cream matricea de confuzie
        conf_matrix = confusion_matrix(self.df['True Label'], self.df['Predicted Label'])

        # Plotam matricea de confuzie
        fig = self.canvas.figure
        fig.clear()

        ax1 = fig.add_subplot(1, 2, 1)
        sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='d', cbar=False, ax=ax1)
        ax1.set_title('Matricea de confuzie\n1 - Vesel, 2 - Frica, 3 - Trist, 4 - Dezgust\n5 - Furie, 6 - Neutru, 7 - Surprins\n')
        ax1.set_xlabel('Predicted Label')
        ax1.set_ylabel('True Label')

        # Cream pie chart pentru match-uri
        ax2 = fig.add_subplot(1, 2, 2)
        match_counts = self.df['Matches'].value_counts()
        labels = ['Non-Match', 'Match']
        ax2.pie(match_counts, labels=labels, autopct='%1.1f%%', colors=['lightcoral', 'lightblue'])
        ax2.set_title('Match vs Non-Match')

        # Salvam figura ca 'graph.png' in folderul 'fisiere_client'
        fig.savefig(os.path.join("fisiere_client", 'graph.png'))

        # Afisare canvas
        self.canvas.draw()
        self.second_button_clicked = True

    def generateDocx(self):
        if not self.first_button_clicked or not self.second_button_clicked:
            QMessageBox.warning(self, "Atentie", "Te rog, apasa mai intai butoanele pentru a citi CSV-ul și a afișa graficul!")
            return

        # Cream un nou document
        doc = Document()

        # Titlul documentului
        doc.add_heading('Clasificarea emotiilor', 0)

        doc.add_heading('\nObiectivul proiectului\n', level=1)

        # Adaugam paragraf explicativ
        doc.add_paragraph(
            'Scopul principal al acestui proiect este de a dezvolta si implementa un sistem de recunoastere a emotiilor umane din imagini, utilizand tehnici de prelucrare a imaginilor si algoritmi de invatare automata. Prin utilizarea unui set de date divers si reprezentativ, proiectul vizeaza antrenarea unui model capabil sa identifice si sa clasifice corect expresiile faciale asociate diferitelor emotii umane. In paralel, se urmareste explorarea eficientei si a limitarilor algoritmului K-Means, utilizat pentru a organiza datele in clustere si interpretarea corespunzatoare a diverselor imagini.\n\nAstfel, obiectivul final consta in dezvoltarea unui sistem robust si precis, capabil sa interpreteze in mod adecvat emotiile umane din imagini, precum si aprofundarea cunostintelor asupra tehnicilor de invatare automata si a inteligentei artificiale in general.\n\n\n'
        )

        # Adaugam tabel cu datele selectate din CSV (coloanele B, C, D)
        doc.add_heading('Rezultatele obtinute dupa antrenarea si testarea modelului:\n', level=1)
        table = doc.add_table(rows=len(self.df) + 1, cols=3)  # Adaugăm doar 3 coloane acum

        # Adăugăm datele din DataFrame în tabel, începând cu a doua coloană (index 1)
        for i in range(len(self.df)):
            for j, col in enumerate(['True Label', 'Predicted Label', 'Matches']):
                value = str(self.df.iloc[i, j + 1]) if pd.notnull(
                    self.df.iloc[i, j + 1]) else 'Matches'  # Highlighted change
                table.cell(i + 1, j).text = value  # Ignorăm prima coloană

        doc.add_paragraph('\n\n')

         # Add generated image
        doc.add_picture(os.path.join("fisiere_client", 'graph.png'), width=Inches(8))

        paragraph = doc.add_paragraph()
        paragraph.add_run('- Matricea de cofuzie:').bold = True
        doc.add_paragraph('Randurile reprezinta etichetele reale, iar coloanele reprezinta etichetele prezise.\n')

        paragraph = doc.add_paragraph()
        paragraph.add_run('- Graficul pentru perfromanta:').bold = True
        doc.add_paragraph('Putem observa de pe grafic rata de clasificare corecta a modelului de unde putem trage concluzii legate de acuratetea modelului.\n\n')

        doc.add_heading('Interpretarea rezultatelor:\n', level=1)
        doc.add_paragraph('Modelul pare sa intampine dificultati in distingerea intre anumite emotii, ceea ce duce la clasificari gresite.\nAnumite clase (de exemplu, angry, disgust, happy) arata predictii corecte, dar altele (fear, sad, surprise) au mai multe clasificari gresite. Precizia generala este relativ scazuta\nPentru a imbunatati performanta modelului, am putea explora caracteristici suplimentare, ajusta parametrii de clusterizare sau lua in considerare algoritmi de clusterizare mai avansati. In plus, cresterea diversitatii si a cantitatii de date de antrenament ar putea imbunatati capacitatea modelului de a generaliza la diferite expresii faciale.')

        doc.add_heading('\n\nConcluzii si observatii\n', level=1)
        doc.add_paragraph('In concluzie performanta obtinuts de modelul nostru a fost satisfacatoare insa nu se poate spune ca e un model extrem de consecvent. Un lucru pe care am putea sa il imbunatatim la setul nostru de date ar fi sa folosim expresii mai categorice si bine definite pentru o antrenare mai buna a modelului.')
        
        # Salvam documentul
        docx_file_path = os.path.join("fisiere_client", "report.docx")
        doc.save(docx_file_path)
        
        # QMessageBox.information(self, "Succes", "Fisierul docx a fost generat cu succes!")
        
        # Trimitem fisierul catre server
        self.send_docx_to_server(docx_file_path)

    def send_docx_to_server(self, docx_file_path):
        HOST = '127.0.0.1'  # Adresa IP
        PORT = 34512  # Port server

        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
            try:
                client_socket.connect((HOST, PORT))
                print("Conectat la server.")

                # Citim fisierul docx byte cu byte
                with open(docx_file_path, 'rb') as file:
                    file_data = file.read()

                # Trimitem intai dimensiunea fisierului
                file_size = len(file_data)
                client_socket.sendall(file_size.to_bytes(4, byteorder='big'))

                # Trimitem datele
                client_socket.sendall(file_data)

                print("Fisier trimis cu succes.")
            except Exception as e:
                print("Eroare:", e)
                QMessageBox.critical(self, "Eroare", f" A aparut o eroare in timp ce fisierul DOCX era trimis catre server: {str(e)}")

In [ ]:
if __name__ == '__main__':
    # Cream o aplicatie Qt
    app = QApplication(sys.argv)
    # Cream fereastra principala
    window = MainWindow()
    # Rulam bucla principala a aplicatiei
    sys.exit(app.exec())